## 第七節　温水暖房

1．適用範囲～4．記号及び単位は省略

### 5. 最大暖房出力

省略

### 6. エネルギー消費量

#### 6.1 消費電力量

$$
\large
E_{E,H,d,t} = E_{E,hs,d,t} + \sum_{i} (E_{E,rad,d,t,i})
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h)  
$E_{E,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機の消費電力量(kWh/h)  
$E_{E,rad,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された放熱器の消費電力量(kWh/h)    

#### 6.2 灯油消費量

$$
\large
E_{K,H,d,t} = E_{K,hs,d,t}
$$

$E_{K,H,d,t}$：日付$d$の時刻$t$における1時間当たりの灯油消費量(MJ/h)  
$E_{K,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機の灯油消費量(MJ/h) 

#### 6.3 ガス消費量

$$
\large
E_{G,H,d,t} = E_{G,hs,d,t}
$$

$E_{G,H,d,t}$：日付$d$の時刻$t$における1時間当たりのガス消費量(MJ/h)  
$E_{G,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機のガス消費量(MJ/h)  

#### 6.4 その他の燃料による一次エネルギー消費量

$$
\large
E_{M,H,d,t} = E_{M,hs,d,t}
$$

$E_{M,H,d,t}$：日付$d$の時刻$t$における1時間当たりのその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{M,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機のその他の燃料による一次エネルギー消費量(MJ/h)  

### 7. 温水暖房用熱源機のエネルギー消費量

#### 7.1 エネルギー消費量

省略

#### 7.2 暖房出力

$$
\large
Q_{out,H,hs,d,t} = min(Q_{dmd,H,hs,d,t}, Q_{max,H,hs,d,t})
$$

$Q_{out,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機の暖房出力(MJ/h)  
$Q_{max,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機の最大暖房出力(MJ/h)  
$Q_{dmd,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機の温水熱需要(MJ/h)  

$$
\large
Q_{dmd,H,hs,d,t} = \sum_{i} (Q_{dmd,H,ln,d,t,i})
$$

$Q_{dmd,H,ln,d,t,i}$：日付$d$の時刻$t$における1時間当たりの放熱系統$i$の温水熱需要(MJ/h)  

#### 7.3 温水供給運転率

$Q_{dmd,H,hs,d,t} > 0$ の場合

$$\large
r_{WS,hs,d,t} = \left\{ \begin{array}{ll}
    r_{WS,ln,d,t,i}&(放熱系統が1系統の場合)\\ 
    1&(放熱系統が2系統以上の場合)\\ 
    \end{array}
\right.
$$

$Q_{dmd,H,hs,d,t} = 0$ の場合

$$\large
r_{WS,hs,d,t} = 0
$$

$r_{WS,hs,d,t}$：日付$d$の時刻$t$における温水暖房用熱源機の温水供給運転率  
$r_{WS,ln,d,t,i}$：日付$d$の時刻$t$における放熱系統$i$の温水供給運転率（ただし、ここでは$i=1$である。）   

#### 7.4 往き温水温度

$\theta_{SW,hs,op,1} > \theta_{SW,hs,op,2} > \theta_{SW,hs,op,3}$  

$\theta_{SW,hs,op,p}$：温水暖房用熱源機の往き温水温度の候補$p$(℃)  

<div style="text-align: center;font-weight: bold;">表　温水暖房用熱源機における往き温水温度の区分及び候補(℃)</div>

<table>
<thead>
<tr>
<th colspan="2" rowspan="2">温水暖房用熱源機の種類</th>
<th colspan="3">往き温水温度の候補の区分 $p$</th>
</tr>
<tr>
<th>1</th>
<th>2</th>
<th>3</th>
</tr>
</thead>

<tbody>
<tr>
<th rowspan="2">石油温水暖房機、石油給湯温水暖房機</th>
<th>従来型</th>
<td>60</td>
<td>-</td>
<td>-</td>
</tr>
<tr>
<th>潜熱回収型</th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th rowspan="2">ガス温水暖房機、ガス給湯温水暖房機</th>
<th>従来型</th>
<td>60</td>
<td>-</td>
<td>-</td>
</tr>
<tr>
<th>潜熱回収型</th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">電気ヒーター温水暖房機、電気ヒーター給湯温水暖房機</th>
<td>60</td>
<td>-</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">電気ヒートポンプ温水暖房機</th>
<td>55</td>
<td>45</td>
<td>35</td>
</tr>

<tr>
<th colspan="2">電気ヒートポンプ・ガス瞬間式併用型湯温水暖房機<br>
（給湯熱源：ガス瞬間式、暖房熱源：電気ヒートポンプ・ガス瞬間式併用） </th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機<br>
（給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：ガス瞬間式）</th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機<br>
（給湯熱源：電気ヒートポンプ・ガス瞬間式併用、<br>
暖房熱源：電気ヒートポンプ・ガス瞬間式併用）</th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">コージェネレーション設備</th>
<td colspan="3">コージェネレーション設備の種類に依る</td>
</tr>

</tbody>
</table>


$$
\large
p_{hs,d,t} = min(p_{ln,d,t,1}, p_{ln,d,t,2}, … p_{ln,d,t,m})
$$

$p_{hs,d,t}$：日付$d$の時刻$t$における温水暖房用熱源機の往き温水温度の区分(℃)  
$p_{ln,d,t,i}$：日付$d$の時刻$t$における放熱系統$i$の要求往き温水温度の区分(℃)  

#### 7.5 戻り温水温度

$$\large
\theta_{RW,hs,d,t} = \left\{ \begin{array}{ll}
    0.0301 \times (T_{dif})^2 - 0.1864 \times T_{dif} + 20&(熱源機の往き温水温度\theta_{SW,hs} = 60)\\ 
    0.0604 \times (T_{dif})^2 - 0.1881 \times T_{dif} + 20&(熱源機の往き温水温度\theta_{SW,hs} = 40)\\ 
    \end{array}
\right.
$$

$$\large
T_{dif,d,t} = \frac{\sum_{i}Q_{dmd,H,ln,d,t,i}}{\sum_{i}Q'_{max,rad,d,t,i} \times  0.027583}
$$

$$\large
Q'_{max,H,rad,d,t,i} = \left\{ \begin{array}{ll}
    Q_{max,H,rad,d,t,i}&(Q_{ln,dmd,d,t,i} > 0)\\ 
    0&(Q_{ln,dmd,d,t,i} = 0)\\ 
    \end{array}
\right.
$$

$T_{dif,d,t}$：日付$d$の時刻$t$における対数平均温度差(℃)  
$Q_{dmd,H,ln,d,t,i}$：日付$d$の時刻$t$における1時間当たりの放熱系統$i$の温水熱需要(MJ/h)  
$Q'_{max,H,rad,d,t,i}$：日付$d$の時刻$t$における1時間当たりの温水熱需要が発生する場合の暖冷房区画$i$に設置された放熱器の最大暖房出力(MJ/h)  
$Q_{max,H,rad,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された放熱器の最大暖房出力(MJ/h)  

### 8. 放熱系統

#### 8.1 温水熱需要

$$
\large
Q_{dmd,H,ln,d,t,i} = Q_{dmd,H,rad,d,t,i} + Q_{loss,pp,d,t,i}
$$

$Q_{dmd,H,rad,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された放熱器の温水熱需要(MJ/h)  
$Q_{loss,pp,d,t,i}$：日付$d$の時刻$t$における1時間当たりの配管$i$の熱損失(MJ/h)  

#### 8.2 温水供給運転率

$$
\large
r_{WS,ln,d,t,i} = r_{WS,rad,d,t,i}
$$

$r_{WS,rad,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$に設置された放熱器の温水供給運転率  

#### 8.3 要求往き温水温度の区分

省略

### 9. 配管

$$\large
Q_{loss,pp,d,t,i} = ((\theta_{SW,d,t} - (\theta_{ex,d,t} \times 0.7 + 20 \times 0.3)) \times L_{pp,ex,i} + (\theta_{SW,d,t} - 20) \times L_{pp,in,i})\\
\large
\times K_{loss,pp,i} \times r_{WS,rad,d,t,i} \times 3600 \times 10^{-6}
$$

$\theta_{SW,d,t}$：日付$d$の時刻$t$における往き温水温度(℃)    
$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)  
$L_{pp,ex,i}$：配管$i$の断熱区画外における長さ(m)  
$L_{pp,in,i}$：配管$i$の断熱区画内における長さ(m)  
$K_{loss,pp,i}$：配管$i$の線熱損失係数(W/mK)  
$r_{WS,rad,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$に設置された放熱器の温水供給運転率  

### 10. 放熱器

#### 10.1 供給熱量

$$\large
Q_{dmd,H,rad,d,t,i} = Q_{T,H,rad,d,t,i} + Q_{loss,rad,d,t,i}
$$

$Q_{T,H,rad,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された放熱器の処理暖房負荷(MJ/h)  
$Q_{loss,rad,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された放熱器の熱損失(MJ/h)  

#### 10.2 消費電力量

日付$d$の時刻$t$における 1 時間当たりの暖冷房区画$i$に設置された放熱器の消費電力量$E_{E,rad,d,t,i}$は、放熱器の種類がパネルラジエーター又は温水床暖房の場合は0とする。

#### 10.3 熱損失

日付$d$の時刻$t$における 1 時間当たりの暖冷房区画$i$に設置された放熱器の熱損失$Q_{loss,rad,d,t,i}$は、放熱器の種類がパネルラジエーター又はファンコンベクターの場合は0とする。

#### 10.4 温水供給運転率

省略

#### 10.5 処理暖房負荷

省略

#### 10.6 最大暖房出力

省略